In [1]:
import pandas as pd
from geojson import Feature, Point, MultiPoint, Polygon, FeatureCollection
import numpy as np
from scipy.spatial import ConvexHull
from scipy import spatial
import geopandas as gpd

# Add building data

In [178]:
buildings_all = gpd.read_file('input_data/all_buildings.shp')
business = gpd.read_file('input_data/business_SF_WGS.shp')
residential_buildings = gpd.read_file('residential_buildings_footprint_2.shp')

# Create Tree

In [179]:
buildings_all['OBJECTID'] = buildings_all.index
X_lat_lon = list(zip(buildings_all['y_lat'],buildings_all['x_lon']))
tree = spatial.KDTree(X_lat_lon)
X_lat_lon_emp = list(zip(business['lat'],business['lng']))

# Find closest building

In [180]:
location = 0
for item in X_lat_lon_emp:
    dist, index_select = tree.query(item, k=1)
    close_to = int(buildings_all.iloc[int(index_select)]['OBJECTID_1'])
    business.loc[int(location),'close_id'] = close_to
    location+=1
    

# Dissolve by closest building and add employee count

In [181]:
new_business = business[['geometry','close_id', 'EMPNUM']]
employee_count = new_business.dissolve(by='close_id', aggfunc='sum')

In [182]:
employee_buildings = buildings_all.join(employee_count, on='OBJECTID_1', how = 'inner', rsuffix='_business')

In [183]:
employee_buildings = employee_buildings.drop('geometry_business',1)

In [184]:
employee_buildings['index_vals'] = employee_buildings['OBJECTID']
employee_buildings['OBJECTID'] = employee_buildings['OBJECTID_1']

# Merge with residential buildings

In [185]:
frames = [residential_buildings, employee_buildings]
result = pd.concat(frames)

In [186]:
subset_result = result[['OBJECTID', 'geometry' ,'SUM_pop', 'EMPNUM']]

# Dissolve based on OBJECTID and sum population and employee count

In [187]:
combined_buildings_sum = subset_result.dissolve(by='OBJECTID', aggfunc='sum')

In [188]:
combined_buildings = buildings_all.join(combined_buildings_sum, on='OBJECTID_1', how = 'inner', rsuffix='_combined')

In [189]:
combined_buildings['residential_pop'] = combined_buildings['SUM_pop']

In [190]:
combined_buildings['SUM_pop'] = combined_buildings.residential_pop.fillna(0) + combined_buildings.EMPNUM.fillna(0)

In [191]:
new_combined_buildings = combined_buildings[['num_floor', 'Area_m2', 'x_lon', 'y_lat', 'SUM_pop', 'residential_pop', 'EMPNUM', 'ELEV_treat']]

In [192]:
new_combined_buildings['lat_lon'] = new_combined_buildings.apply(lambda row: (row['y_lat'], row['x_lon']), axis=1)

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
new_combined_buildings.fillna(value =0, inplace=True) 

In [9]:
new_combined_buildings['SUM_pop_residential'] = new_combined_buildings['residential_pop']
new_combined_buildings['SUM_pop_commercial'] = new_combined_buildings['EMPNUM']
new_combined_buildings = new_combined_buildings.drop(['residential_pop', 'EMPNUM'],1)

,Unnamed: 0,num_floor,Area_m2,x_lon,y_lat,SUM_pop,ELEV_treat,lat_lon,SUM_pop_residential,SUM_pop_commercial
0,0,5,1805.444103,-122.479750,37.751639,38.201872,120.883926,"(37.751638988499998, -122.479749637)",38.201872,0.0
1,1,3,1039.831840,-122.478203,37.751398,24.891947,121.544205,"(37.751398283199997, -122.478203326)",24.891947,0.0
2,2,3,1659.337587,-122.497269,37.755927,44.208846,48.054523,"(37.755927296000003, -122.49726864900001)",44.208846,0.0
3,3,2,1451.413216,-122.500518,37.756384,34.610670,32.513432,"(37.756384032, -122.50051803700001)",34.610670,0.0
4,4,6,2434.756642,-122.465517,37.755957,71.720541,139.534668,"(37.755956715499998, -122.465516691)",71.720541,0.0
5,5,3,1023.404982,-122.471378,37.737857,17.584762,89.404564,"(37.737857069699999, -122.471377797)",17.584762,0.0
6,7,7,3384.219715,-122.409811,37.805659,150.785856,7.748829,"(37.8056594508, -122.40981103)",145.785856,5.0
7,8,5,3346.300978,-122.404720,37.805287,778.000000,2.959130,"(37.805286603600003, -122.40472037399999)",0.000000,778.0
8,9,3,718.554079,-122.402049,37.804590,2.000000,1.967958,"(37.804589838600002, -122.40204913300001)",0.000000,2.0
9,10,10,3691.850332,-122.407431,37.802071,141.479483,47.383457,"(37.8020706549, -122.40743122000001)",137.479483,4.0


In [16]:
new_combined_buildings.to_csv('combined_buildings_2.csv')